<h1><center>Laboratorio 9: Optimización de modelos 💯</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Ignacio Meza, Gabriel Iturra
- Auxiliar: Sebastián Tinoco
- Ayudante: Arturo Lazcano, Angelo Muñoz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Nicolás Becerra
- Nombre de alumno 2: Simón Sanfeliú


## Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.

## Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer material del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Optimizar modelos usando `optuna`
- Recurrir a técnicas de *prunning*
- Forzar el aprendizaje de relaciones entre variables mediante *constraints*
- Fijar un pipeline con un modelo base que luego se irá optimizando.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

### **Link de repositorio de GitHub:** https://github.com/SimonSanfeliu/MDS7202-BS/tree/L9

# Importamos librerias útiles

In [54]:
!pip install -qq xgboost optuna

# 1. El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Como usted tuvo un rendimiento sobresaliente en el proyecto de caracterización de datos, Fiu lo contrata como *data scientist* de su emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [55]:
import pandas as pd
import numpy as np
from datetime import datetime

df = pd.read_csv('sales.csv')
df['date'] = pd.to_datetime(df['date'])

df.head()

C:\Users\sanfe\AppData\Local\Temp\ipykernel_10616\3184305967.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df['date'])


,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696


## 1.1 Generando un Baseline (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p>

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad.
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas.
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas.
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios.
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio?
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`?
7. Guarde ambos modelos en un archivo .pkl (uno cada uno)

In [56]:
# Función para FunctionTransformer
def extract_date(df):
    """
    Agregar docstring 
    """
    # Revisando datos
    assert type(df) == pd.DataFrame
    assert "date" in df.columns

    # Generando las columnas
    df = df.assign(
        year = [d.year for d in df["date"].dt.date],
        month = [d.month for d in df["date"].dt.date],
        day = [d.day for d in df["date"].dt.date]
    )

    # Transformándolas en categorías
    df = df.astype(
        {
            "day": "category",
            "month": "category",
            "year": "category"
        }
    )

    return df

# Función para transformación logarítmica
def to_log(df_s):
    """
    Agregar docstring 
    """
    # Revisando datos
    assert type(df_s) == pd.DataFrame

    # Transformando los datos de la serie a escala logarítmica
    df_s = df_s.apply(lambda x: np.log(x + 1))
    return df_s

In [69]:
# Obteniendo librerías necesarias
from sklearn.model_selection import train_test_split 
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer, MinMaxScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error

# Definiendo la semilla
RANDOM_STATE = 42

# Separando el conjunto de datos
X_train, X_rest, y_train, y_rest = train_test_split(df, df["quantity"], test_size=.3, random_state=RANDOM_STATE)
X_val, X_test, y_val, y_test = train_test_split(X_rest, y_rest, test_size=.33, random_state=RANDOM_STATE)
X_train.drop(columns=["id", "quantity"], inplace=True)
X_val.drop(columns=["id", "quantity"], inplace=True)
X_test.drop(columns=["id", "quantity"], inplace=True)

# Separando los datos en numéricos y categóricos
num_cols = X_train.corr(numeric_only=True).columns.to_list()
cat_cols = [col for col in X_train.columns if not col in num_cols]
cat_cols.append("year")  # Agregando las nuevas columnas que se obtendrán del FunctionTransformer
cat_cols.append("month")
cat_cols.append("day")

# Atributos numéricos
num_pipe = Pipeline([
                ('Logaritmic scaler', FunctionTransformer(to_log, feature_names_out='one-to-one')),
                ('MinMax scaler', MinMaxScaler())
            ])
# Atributos categóricos
cat_pipe = Pipeline([
    ('Encoder', OneHotEncoder(sparse_output=False))
])

# Creando ColumnTransformer
ctrans = ColumnTransformer(
        transformers=[
            ("Categorico", cat_pipe, cat_cols),
            ("Numerico", num_pipe, num_cols),
        ],
        remainder="passthrough",
        verbose_feature_names_out=False
)
ctrans.set_output(transform='pandas')

ColumnTransformer(remainder='passthrough',
                  transformers=[('Categorico',
                                 Pipeline(steps=[('Encoder',
                                                  OneHotEncoder(sparse_output=False))]),
                                 ['date', 'city', 'shop', 'brand', 'container',
                                  'capacity', 'year', 'month', 'day']),
                                ('Numerico',
                                 Pipeline(steps=[('Logaritmic scaler',
                                                  FunctionTransformer(feature_names_out='one-to-one',
                                                                      func=<function to_log at 0x0000023224339A20>)),
                                                 ('MinMax scaler',
                                                  MinMaxScaler())]),
                                 ['lat', 'long', 'pop', 'price'])],
                  verbose_feature_names_out=False)

In [70]:
# Pipeline de entrenamiento 1
pipe_train1 = Pipeline([
    ("Date extraction", FunctionTransformer(extract_date)),
    ("Scaling", ctrans),
    ("Classifier", DummyRegressor())
])

# Entrenando
model1 = pipe_train1.fit(X_train, y_train)

# Prediciendo
y_pred1 = model1.predict(X_val)

# MAE
mae1 = mean_absolute_error(y_val, y_pred1)
print(f"MAE sobre conjunto de validación: {mae1}")

MAE sobre conjunto de validación: 13308.134750658153


> El `mean_absolute_error` es una medida de diferencia entre el valor predicho y el valor real, siendo ésta el error absoluto medio entre ambos. Lo ideal es que esta métrica sea lo más cercana a 0 posible, ya que implicaría que el valor predicho no dista mucho del real, por lo que se tendría una buena predicción. 

> Dado que el valor obtenido del MAE es alrededor de 13000, se tiene que para la pipeline generada con `DummyRegressor` es de muy baja calidad, ya que los valores predichos están muy alejados de los valores reales. Así, se tiene que el primer modelo generado es muy malo para la predicción de la demanda de cantidad de ventas, por lo que la métrica estaría diciendo que este modelo no es bueno para el negocio.

In [71]:
import xgboost as xgb

# Pipeline de entrenamiento 2
pipe_train2 = Pipeline([
    ("Date extraction", FunctionTransformer(extract_date)),
    ("Scaling", ctrans),
    ("Classifier", xgb.XGBRegressor())
])

# Entrenando
model2 = pipe_train2.fit(X_train, y_train)

# Prediciendo
y_pred2 = model2.predict(X_val)

# MAE
mae2 = mean_absolute_error(y_val, y_pred2)
print(f"MAE sobre conjunto de validación: {mae2}")

MAE sobre conjunto de validación: 2500.3221446955317


> Al cambiar el `DummyRegressor` por el `XGBRegressor`, se tiene una mejora instantánea en la métrica, teniendo ahora un MAE de cerca de 2500, lo que es mucho más cercano a 0 que 13000. Este valor sigue indicando que el clasificador es malo para el negocio, pero muestra una inmediata mejora con el anterior.

In [72]:
import os
import joblib

# Creando la carpeta de los modelos si es que no existe
if not os.path.exists("models"):
    os.mkdir("models")

# Guardando los archivos en pkls separados
joblib.dump(model1, "models/dummy_model.pkl")
joblib.dump(model2, "models/default_xgb_model.pkl")

['models/default_xgb_model.pkl']

## 1.2 Forzando relaciones entre parámetros con XGBoost (1.0 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo.

Vuelva a entrenar el `Pipeline`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo?

Nuevamente, guarde su modelo en un archivo .pkl

Nota: Para realizar esta parte, debe apoyarse en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>.

Hint: Para implementar el constraint, se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

In [73]:
# Pipeline de entrenamiento 3
pipe_train3 = Pipeline([
    ("Date extraction", FunctionTransformer(extract_date)),
    ("Scaling", ctrans),
    ("Classifier", xgb.XGBRegressor(monotone_constraints={'price': -1}))
])

# Entrenando
model3 = pipe_train3.fit(X_train, y_train)

# Prediciendo
y_pred3 = model3.predict(X_val)

# MAE
mae3 = mean_absolute_error(y_val, y_pred3)
print(f"MAE sobre conjunto de validación: {mae3}")

MAE sobre conjunto de validación: 2707.9117589321568


> Explicación XD

In [75]:
# Guardando el modelo
joblib.dump(model3, "models/constraint_xgb_model.pkl")

['models/constraint_xgb_model.pkl']

## 1.3 Optimización de Hiperparámetros con Optuna (2.0 puntos)

<p align="center">
  <img src="https://media.tenor.com/fmNdyGN4z5kAAAAi/hacking-lucy.gif">
</p>

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se le pide:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)
- Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados?
- Fije el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

In [ ]:
# Inserte su código acá

## 1.4 Optimización de Hiperparámetros con Optuna y Prunners (1.7)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento?
- Utilizar `optuna.integration.XGBoostPruningCallback` como método de **Prunning**
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

In [ ]:
# Inserte su código acá

## 1.5 Visualizaciones (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/F-LgB1xTebEAAAAd/look-at-this-graph-nickelback.gif">
</p>


Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

- Gráfico de historial de optimización
- Gráfico de coordenadas paralelas
- Gráfico de importancia de hiperparámetros

Comente sus resultados: ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados? ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas? ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo?

In [ ]:
# Inserte su código acá

## 1.6 Síntesis de resultados (0.3)

Finalmente, genere una tabla resumen del MAE obtenido en los 5 modelos entrenados (desde Baseline hasta XGBoost con Constraints, Optuna y Prunning) y compare sus resultados. ¿Qué modelo obtiene el mejor rendimiento? 

Por último, cargue el mejor modelo, prediga sobre el conjunto de test y reporte su MAE. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto?

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media.tenor.com/8CT1AXElF_cAAAAC/gojo-satoru.gif">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>